In [1]:
import os
import yaml
import time
import numpy as np
import pandas as pd
import shelve
import sys
sys.path.append('../')
from geecs_python_api.controls.interface import GeecsDatabase
from geecs_python_api.controls.devices.geecs_device import GeecsDevice

GeecsDevice.exp_info = GeecsDatabase.collect_exp_info("Undulator")

In [2]:
# Jet_Stages_xopt = OptimizationControl(device_name='U_ESP_JetXYZ',variable='Position.Axis 1', bounds=[5,6])
hexapod = GeecsDevice('U_Hexapod')
aline3=GeecsDevice('UC_ALineEBeam3')
aline3.get("maxcounts")

748.0

In [3]:
#define a "yaml file" with no entries for variables. These will be filled in used the "bella_config" yamml file
YAML = """
xopt:
    dump_file: dump.yaml
generator:
  name: neldermead
  initial_point: {"HexY": 18.0, "HexZ": 0.1,"HexV":0.011,"HexW":-0.21}
  adaptive: true
  xatol: 0.0001
  fatol: 0.0001  

evaluator:
    function: __main__.geecs_measurement

vocs:
    variables:
        {}
    objectives: {f: "MAXIMIZE"}

"""

yaml_output = yaml.safe_load(YAML)

objs={'HexY':{"GEECS_Object":hexapod,'variable':"ypos","bounds":[17,19]},
     'HexZ':{"GEECS_Object":hexapod,'variable':"zpos","bounds":[-.4,1]},
      'HexV':{"GEECS_Object":hexapod,'variable':"vangle","bounds":[-.2,.2]},
     'HexW':{"GEECS_Object":hexapod,'variable':"wangle","bounds":[-.4,.1]}}

for tag in objs.keys():
    yaml_output['vocs']['variables'][tag]=objs[tag]['bounds']
    
print(yaml_output)

{'xopt': {'dump_file': 'dump.yaml'}, 'generator': {'name': 'neldermead', 'initial_point': {'HexY': 18.0, 'HexZ': 0.1, 'HexV': 0.011, 'HexW': -0.21}, 'adaptive': True, 'xatol': 0.0001, 'fatol': 0.0001}, 'evaluator': {'function': '__main__.geecs_measurement'}, 'vocs': {'variables': {'HexY': [17, 19], 'HexZ': [-0.4, 1], 'HexV': [-0.2, 0.2], 'HexW': [-0.4, 0.1]}, 'objectives': {'f': 'MAXIMIZE'}}}


In [58]:
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
#simulate objective using set values. 
def simulate_objective(input_dict):
    v1=gaussian(float(input_dict['JetX']),5.5,.3)+np.random.normal(0,.05,1)[0]
    v2=gaussian(float(input_dict['JetY']),-7,.5)+np.random.normal(0,.05,1)[0]
#     v3=gaussian(float(input_dict['JetZ']),0.4,.1)+np.random.normal(0,.05,1)[0]
#     v4=gaussian(float(input_dict['Pressure']),0.9,.25)+np.random.normal(0,.05,1)[0]
    
#     v5=0.8*gaussian(float(input_dict['JetBlade']),0.5+.2,.3)+np.random.normal(0,.05,1)[0]
#     v6=0.8*gaussian(float(input_dict['LampTiming']),0.2+.2,.5)+np.random.normal(0,.05,1)[0]
#     v7=0.8*gaussian(float(input_dict['JetZ']),0.4+.2,.1)+np.random.normal(0,.05,1)[0]
#     v8=0.8*gaussian(float(input_dict['Pressure']),0.9-.2,.25)+np.random.normal(0,.05,1)[0]
    res=v1*v2
    res=float("%.6f" % res)
    return res

In [62]:
simulate_objective({"JetX":5.5,"JetY":-7})

1.026686

In [5]:

def geecs_measurement(input_dict):

    for i in list(input_dict.keys()):
        set_val=float(input_dict[i])
        
#         print("set {:d} to {:.4f}".format(i, set_val))
#         print(f"set {i:d} to {set_val:.4f}")
        print('set '+str(i)+' to '+str(set_val))
        objs[i]["GEECS_Object"].set(objs[i]["variable"],set_val)
        time.sleep(1)

   
    outs=[]
    counter=0
    while counter<11:
        #val=simulate_objective(input_dict)
        val=aline3.get("meancounts")
        outs.append(val)
        counter=counter+1
        
    print(outs)

# this one is good for aligning to a crosshair
    return {'f': abs(np.median(outs))}


In [6]:
geecs_measurement({"HexY": 18.45, "HexZ": 0.6,"HexV":-0.25,"HexW":-0.15})

set HexY to 18.45
set HexZ to 0.6
set HexV to -0.25
set HexW to -0.15
[1.027922, 1.025834, 1.029529, 0.996417, 1.007706, 1.002763, 1.008977, 1.019941, 1.004825, 1.015082, 0.999047]


{'f': 1.008977}

In [2]:
from xopt.evaluator import Evaluator
from xopt.generators.bayesian import ExpectedImprovementGenerator
from xopt import Xopt

X = Xopt(config=yaml_output)
X

NameError: name 'yaml_output' is not defined

NameError: name 'X' is not defined

In [148]:
import torch
import matplotlib.pyplot as plt
n_steps = 100

# test points for plotting
#test_x = torch.linspace(*X.vocs.bounds.flatten(),50).double()

for i in range(n_steps):
    print('step'+str(i))
    X.step()

step0
set HexV to 0.011
set HexW to -0.21
set HexY to 18.0
set HexZ to 0.1
[4.312755, 4.312316, 4.272238, 4.480049, 4.333275, 4.419974, 4.339303, 4.453934, 4.495142, 4.380888, 4.456098]
step1
set HexV to 0.01155
set HexW to -0.21
set HexY to 18.0
set HexZ to 0.1
[4.44344, 4.477808, 4.469006, 4.415748, 4.490917, 4.479268, 4.420791, 4.425413, 4.456923, 4.519797, 4.490106]
step2
set HexV to 0.011
set HexW to -0.2205
set HexY to 18.0
set HexZ to 0.1
[4.453995, 4.422794, 4.40425, 4.465443, 4.427777, 4.359433, 4.431427, 4.438807, 4.500234, 4.433764, 4.501797]
step3
set HexV to 0.011
set HexW to -0.21
set HexY to 18.900000000000002
set HexZ to 0.1
[4.269051, 4.216743, 4.170835, 4.168436, 4.174608, 4.129409, 4.13044, 4.234499, 4.192436, 4.073634, 4.153895]
step4
set HexV to 0.011
set HexW to -0.21
set HexY to 18.0
set HexZ to 0.10500000000000001
[4.597898, 4.589222, 4.540071, 4.595427, 4.541724, 4.62282, 4.626928, 4.577567, 4.506655, 4.568228, 4.580775]
step5
set HexV to 0.011274999999999997
s

UnboundLocalError: local variable 'result' referenced before assignment

In [100]:
X.data

,JetX,JetY,f,xopt_runtime,xopt_error
1,5.100000,-6.100000,0.082782,4.052208,False
2,5.355000,-6.100000,0.161452,1.534378,False
3,5.100000,-6.405000,0.202245,2.541570,False
4,5.355000,-6.405000,0.460304,1.540118,False
5,5.482500,-6.557500,0.657130,2.538185,False
6,5.227500,-6.862500,0.605988,2.395879,False
7,5.610000,-7.015000,0.909877,2.795503,False
8,5.865000,-7.320000,0.406876,2.392350,False
9,5.865000,-6.710000,0.354089,2.703170,False
10,5.386875,-6.824375,0.904952,2.795647,False
